# Lasso
Lasso ( menor operador de seleção e encolhimento absoluto) é um método de análise de regressão que executa seleção e regularização de variáveis para aumentar a precisão e a capacidade de interpretação do modelo estatístico produzido. 

A regressão de Lasso difere da regressão de rebordo de uma maneira que usa valores absolutos na função de penalidade, em vez de quadrados. Isto leva a penalizar (ou equivalentemente restringir a soma dos valores absolutos das estimativas) valores que fazem com que algumas das estimativas dos parâmetros sejam exatamente zero. Quanto maior a penalidade aplicada, mais as estimativas são reduzidas para o zero absoluto. Isso resulta na seleção de variáveis de n variáveis dadas.

## Pontos importantes
- As suposições desta regressão são as mesmas que a regressão do quadrado mínimo, exceto que a normalidade não deve ser assumida
- Ele reduz os coeficientes para zero (exatamente zero), o que certamente ajuda na seleção de recursos
- Este é um método de regularização e usa a regularização L1
- Se o grupo de preditores estiver altamente correlacionado, o laço pega apenas um deles e encurta os outros para zero

## Bibliotecas e Dados

In [54]:
from sklearn.linear_model import Lasso
import numpy as np
import pandas as pd
import logging
import time
from sklearn import metrics

logger = logging.getLogger('spam_application')
logger.setLevel(logging.DEBUG)

def move_column_to_front(data, column_name):
    # Move selected column to 0 position. can be used to isolate the Y column.
    
    mid = data[column_name]
    data.drop(labels=[column_name], axis=1,inplace = True)
    data.insert(0, column_name, mid)
    del mid
    return data


def load_and_prepare_data(training_set_number=2500):
    # prepare data and return x_train, x_test, y_train and y_test
    
    # loading data
    data = 'SkillCraft.csv'
    df = pd.read_csv(data)
    
    # identifying Y column name
    y_column = 'APM'
    
    # training_set_number must be in interval [0, len(df)]. otherwise, will be 75%.
    if training_set_number >= len(df) or training_set_number <= 0:
        training_set_number = int(0.75 * len(df))
    
    # deleting GameID
    logger.info('creating an instance of auxiliary_module.Auxiliary')
    cleaned_data = df.drop('GameID', 1)

    # move APM column to front
    cleaned_data = move_column_to_front(cleaned_data, y_column)

    # Checando se existem valores nulos
    null_values = cleaned_data.isnull().values.any()
    assert null_values == False
    
    # isolating training set
    train=cleaned_data.iloc[:training_set_number]
    
    # isolating test set
    test=cleaned_data[training_set_number:]


    x_train = train.iloc[:, 1:].values
    x_test = test.iloc[:, 1:].values
    y_train = train[y_column].values
    y_test = test[y_column].values
    
    return x_train, x_test, y_train, y_test

# Divide os dados em conjunto de treinamento e em conjunto de teste
features_train, features_test, apm_train, apm_test = load_and_prepare_data()

In [55]:
def train_regression(lasso, features_train, features_test, apm_train, apm_test):
    # Treinamento
    start = time.time()
    lasso.fit(features_train, apm_train)
    end = time.time()
    print ("Training time: {}ms".format(int((end-start)*1000*1000)/1000))

    # Predict
    predicted = lasso.predict(features_test)

    # print statistics
    print("Explained variance score: {}%".format(int((metrics.explained_variance_score(apm_test, predicted))*10000)/100))
    print("R2 score: {}%".format(int((metrics.r2_score(apm_test, predicted))*10000)/100))
    print("Mean absolute error: {}".format(int((metrics.mean_absolute_error(apm_test, predicted)*1000))/1000))
    print("Mean squared error: {}".format(int((metrics.mean_squared_error(apm_test, predicted)*1000))/1000))
    print("Median absolute error: {}".format(int((metrics.median_absolute_error(apm_test, predicted)*1000))/1000))

# Descrição dos Parâmetros
- alpha é a constante que multiplica L1 e o intervalo vao de 0 (zero) a 1.0, sendo 0 (zero) o equivalente ao algoritmo de Regressão Linear
- fit_intercept é um parametro booleano que indica se será calculada a interceptação para este modelo.
- normilize é um booleano que indica se os regressores X serão normalizados.
- precompute serve para acelerar os calculos caso seja igual a True
- max_iter é o número máximo de interações 
- tol é a tolerancia para otimização, se as atualizações forem menores que tol, para de atualizar.

In [43]:
# Teste com os valores default 
lasso = Lasso(alpha=1.0,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        copy_X=True,
        max_iter=1000,
        tol=0.0001,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='cyclic')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 6.366ms
Explained variance score: 62.59%
R2 score: 62.25%
Mean absolute error: 20.247
Mean squared error: 934.876
Median absolute error: 14.209


In [44]:
# Teste com os valores default e alpha=0.01
lasso = Lasso(alpha=0.01,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        copy_X=True,
        max_iter=1000,
        tol=0.0001,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='cyclic')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 5.746ms
Explained variance score: 92.49%
R2 score: 92.48%
Mean absolute error: 10.105
Mean squared error: 186.028
Median absolute error: 7.984


In [45]:
# Teste com os valores default e alpha=0.00001
lasso = Lasso(alpha=0.00001,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        copy_X=True,
        max_iter=1000,
        tol=0.0001,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='cyclic')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 12.865ms
Explained variance score: 97.09%
R2 score: 97.09%
Mean absolute error: 5.702
Mean squared error: 71.989
Median absolute error: 3.935


In [50]:
# Teste com os valores default e alpha=0.00001, max_inter=1000000, tol=0.00000001, selection='random'
lasso = Lasso(alpha=0.00001,
        fit_intercept=True,
        normalize=True,
        precompute=False,
        copy_X=True,
        max_iter=1000000,
        tol=0.00000001,
        warm_start=False,
        positive=True,
        random_state=None,
        selection='random')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 10.833ms
Explained variance score: 96.86%
R2 score: 96.86%
Mean absolute error: 5.893
Mean squared error: 77.581
Median absolute error: 4.24


In [51]:
# Teste com os valores default e alpha=0.00001, max_inter=1000000, tol=0.00000001, fit_intercept=False, positive=False, selection='random'
lasso = Lasso(alpha=0.00001,
        fit_intercept=False,
        normalize=True,
        precompute=False,
        copy_X=True,
        max_iter=1000000,
        tol=0.00000001,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='random')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 179.704ms
Explained variance score: 96.91%
R2 score: 96.91%
Mean absolute error: 6.198
Mean squared error: 76.458
Median absolute error: 4.631


In [52]:
# Teste com os valores default e alpha=0.00001, max_inter=1000000, tol=0.00000001, fit_intercept=False, positive=False, selection='random', positive=True
lasso = Lasso(alpha=0.00001,
        fit_intercept=False,
        normalize=True,
        precompute=False,
        copy_X=True,
        max_iter=1000000,
        tol=0.00000001,
        warm_start=False,
        positive=True,
        random_state=None,
        selection='random')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 50.602ms
Explained variance score: 91.02%
R2 score: 91.01%
Mean absolute error: 12.116
Mean squared error: 222.473
Median absolute error: 10.877


In [53]:
# Teste com os valores default e alpha=0.00001, max_inter=1000000, tol=0.00000001, fit_intercept=False, positive=False, selection='cyclic', positive=True
lasso = Lasso(alpha=0.00001,
        fit_intercept=False,
        normalize=True,
        precompute=False,
        copy_X=True,
        max_iter=1000000,
        tol=0.00000001,
        warm_start=False,
        positive=True,
        random_state=None,
        selection='cyclic')
train_regression(lasso, features_train, features_test, apm_train, apm_test)

Training time: 42.699ms
Explained variance score: 91.02%
R2 score: 91.01%
Mean absolute error: 12.116
Mean squared error: 222.473
Median absolute error: 10.877
